<div style="background-color: #ff87b2; padding: 20px; border-radius: 10px;">
<h3>Atividade 4:</h3>

Para o estudo de caso do Predição de Cargas de Trabalho para Juízes cujo objetivo prever o volume de trabalho em diferentes tribunais com base em fatores como tamanho da população, tipos de casos comuns na região, etc.

Execute a otimização de hiperparâmetros para o : BayesianRidge, RandomForestRegressor, GradientBoostingRegressor, e MLPRegressor.

Monte um DataFrame comparativo mostrando as métricas MAE, MSE, RMSE, R2 de cada modelo.

Poste no AVA o Jupyter Notebook ou o link para o repositório GitHub.
  
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

import joblib

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import r2_score


In [3]:
df_normalizado = pd.read_csv("../data/processed/predicao_cargas_trabalho_normalizado.csv")

X = df_normalizado.loc[:, df_normalizado.columns != 'casos_pendentes']
y = df_normalizado["casos_pendentes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# RandomForestRegressor+GridSearchCV

In [4]:
pipeline = Pipeline([
    ('model', RandomForestRegressor())
])


# Definir a grade de parâmetros
param_grid = {
    'model__n_estimators':[1, 3, 80, 54, 44, 34, 99, 21],
    'model__max_features':["sqrt", "log2", "float", "int"],
    'model__max_samples': [0.1, 0.4, 0.7, 0.9, 1.0]
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=1; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=1; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=3; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=3; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=1; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=1; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=1; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=3; total time=   0.0s
[CV] END model__max_features=sqrt, model__max_samples=0.1, model__n_estimators=3; total time=   0.0s
[CV] END model__max_features

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'model__max_features': ['sqrt', 'log2', 'float',
                                                 'int'],
                         'model__max_samples': [0.1, 0.4, 0.7, 0.9, 1.0],
                         'model__n_estimators': [1, 3, 80, 54, 44, 34, 99, 21]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [5]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__max_features': 'sqrt', 'model__max_samples': 1.0, 'model__n_estimators': 99}


In [6]:
# Ajustar o modelo com os melhores parâmetros
modelRidge = grid_search.best_estimator_
modelRidge.fit(X_train, y_train)

predictions = modelRidge.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.06410925110977612
MSE: 0.007081373531393507
RMSE: 0.08415089738911587
R2: 0.718451401126845


In [7]:
metric_model = {
    'MODEL': 'RandomForestRegressor+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result = []
models_result.append(metric_model)

# BayesianRidge+GridSearchCV

In [8]:
pipeline = Pipeline([
    ('model', BayesianRidge())
])


# Definir a grade de parâmetros
param_grid = {
    'model__alpha_1':[1e-4, 0.4, 1.5, 0.1, 3.0],
    'model__compute_score':[False, True],
    'model__tol': [1e-7, 0.2, 0.5, 1.5, 2.0],
    'model__lambda_2': [1e-8, 0.2, 0.5, 1.5, 2.0]
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=1e-07; total time=   0.0s[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=1e-07; total time=   0.0s

[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=1e-07; total time=   0.0s
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=1e-07; total time=   0.0s
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=1e-07; total time=   0.0s
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=0.2; total time=   0.0s
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=0.2; total time=   0.0s
[CV] END model__alpha_1=0.0001, model__compute_score=False, model__lambda_2=1e-08, model__tol=0.2; t

GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', BayesianRidge())]),
             n_jobs=-1,
             param_grid={'model__alpha_1': [0.0001, 0.4, 1.5, 0.1, 3.0],
                         'model__compute_score': [False, True],
                         'model__lambda_2': [1e-08, 0.2, 0.5, 1.5, 2.0],
                         'model__tol': [1e-07, 0.2, 0.5, 1.5, 2.0]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [9]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__alpha_1': 0.0001, 'model__compute_score': False, 'model__lambda_2': 1e-08, 'model__tol': 1e-07}


In [10]:
# Ajustar o modelo com os melhores parâmetros
modelRidge = grid_search.best_estimator_
modelRidge.fit(X_train, y_train)

predictions = modelRidge.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.05660617995807335
MSE: 0.005557543781376615
RMSE: 0.0745489354811765
R2: 0.7790374059656632


In [11]:
metric_model = {
    'MODEL': 'BayesianRidge+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

# GradientBoostingRegressor+GridSearchCV

In [12]:
pipeline = Pipeline([
    ('model', GradientBoostingRegressor())
])


# Definir a grade de parâmetros
param_grid = {
    'model__n_estimators':[2, 200, 45, 350],
    'model__max_depth': [1, 0.3, 5, 10, 20],
    'model__criterion': ["friedman_mse", "squared_error"]
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=2; total time=   0.0s[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=2; total time=   0.0s

[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=2; total time=   0.0s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=2; total time=   0.0s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=2; total time=   0.0s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=45; total time=   0.2s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=45; total time=   0.2s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=45; total time=   0.3s
[CV] END model__criterion=friedman_mse, model__max_depth=1, model__n_estimators=45; total time=   0.2s
[CV] END model__

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model', GradientBoostingRegressor())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['friedman_mse', 'squared_error'],
                         'model__max_depth': [1, 0.3, 5, 10, 20],
                         'model__n_estimators': [2, 200, 45, 350]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [13]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__criterion': 'friedman_mse', 'model__max_depth': 1, 'model__n_estimators': 200}


In [14]:
# Ajustar o modelo com os melhores parâmetros
modelRidge = grid_search.best_estimator_
modelRidge.fit(X_train, y_train)

predictions = modelRidge.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.05709165290198007
MSE: 0.005651198075626452
RMSE: 0.0751744509499501
R2: 0.7753138013277213


In [15]:
metric_model = {
    'MODEL': 'GradientBoostingRegressor+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

# MLPRegressor+GridSearchCV

In [16]:
pipeline = Pipeline([
    ('model', MLPRegressor())
])


# Definir a grade de parâmetros
param_grid = {
    'model__solver':["lbfgs", "sgd", "adam"],
    'model__activation':["identity", "logistic", "tanh", "relu"],
    'model__alpha': [0.0001, 0.05, 0.1, 1.5, 5.0],
    'model__learning_rate': ["constant", "invscaling", "adaptive"]
}

grid_search = GridSearchCV(pipeline, param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True,
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=lbfgs; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=lbfgs; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=lbfgs; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=lbfgs; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=lbfgs; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=adam; total time=   0.2s
[CV] END model__activation=identity, model__alpha=0.0001, model__learning_rate=constant, model__solver=adam; total time=   0.3s
[CV] END model__activation=identity,

/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=lbfgs; total time=   0.6s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=constant, model__solver=sgd; total time=   2.4s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=sgd; total time=   0.2s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=sgd; total time=   0.3s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=sgd; total time=   0.2s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=sgd; total time=   0.2s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=sgd; total time=   0.2s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=invscaling, model__solver=lbfgs; total tim

/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=adaptive, model__solver=sgd; total time=   2.4s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=adaptive, model__solver=sgd; total time=   2.5s
[CV] END model__activation=identity, model__alpha=1.5, model__learning_rate=adaptive, model__solver=sgd; total time=   2.4s
[CV] END model__activation=identity, model__alpha=5.0, model__learning_rate=constant, model__solver=lbfgs; total time=   0.5s
[CV] END model__activation=identity, model__alpha=5.0, model__learning_rate=constant, model__solver=lbfgs; total time=   0.5s
[CV] END model__activation=identity, model__alpha=5.0, model__learning_rate=constant, model__solver=lbfgs; total time=   0.5s
[CV] END model__activation=identity, model__alpha=5.0, model__learning_rate=constant, model__solver=lbfgs; total time=   0.5s
[CV] END model__activation=identity, model__alpha=5.0, model__learning_rate=constant, model__solver=adam; total time=   0.8s

...

/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=constant, model__solver=sgd; total time=   2.8s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=invscaling, model__solver=sgd; total time=   0.4s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=invscaling, model__solver=lbfgs; total time=   0.9s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=invscaling, model__solver=sgd; total time=   0.6s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=invscaling, model__solver=adam; total time=   1.1s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=invscaling, model__solver=adam; total time=   1.3s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=adaptive, model__solver=lbfgs; total time=   1.3s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=adaptive, model__solver=lbfgs; total time=   1.1s
[CV] END model__activa

/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/t319008/miniconda3/envs/unidade3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=adaptive, model__solver=sgd; total time=   2.5s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=adaptive, model__solver=sgd; total time=   2.7s
[CV] END model__activation=relu, model__alpha=5.0, model__learning_rate=adaptive, model__solver=sgd; total time=   2.8s


GridSearchCV(cv=5, estimator=Pipeline(steps=[('model', MLPRegressor())]),
             n_jobs=-1,
             param_grid={'model__activation': ['identity', 'logistic', 'tanh',
                                               'relu'],
                         'model__alpha': [0.0001, 0.05, 0.1, 1.5, 5.0],
                         'model__learning_rate': ['constant', 'invscaling',
                                                  'adaptive'],
                         'model__solver': ['lbfgs', 'sgd', 'adam']},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=2)

In [17]:
# Imprimir os melhores parâmetros
print("Melhores parâmetros: ", grid_search.best_params_)

Melhores parâmetros:  {'model__activation': 'identity', 'model__alpha': 0.0001, 'model__learning_rate': 'constant', 'model__solver': 'lbfgs'}


In [18]:
# Ajustar o modelo com os melhores parâmetros
modelRidge = grid_search.best_estimator_
modelRidge.fit(X_train, y_train)

predictions = modelRidge.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R2:', r2_score(y_test, predictions))

MAE: 0.05661300876628158
MSE: 0.005559774304394472
RMSE: 0.07456389410696354
R2: 0.7789487225163791


In [19]:
metric_model = {
    'MODEL': 'MLPRegressor+GridSearchCV',
    'MAE': metrics.mean_absolute_error(y_test, predictions),
    'MSE': metrics.mean_squared_error(y_test, predictions),
    'RMSE': np.sqrt(metrics.mean_squared_error(y_test, predictions)),
    'R2': r2_score(y_test, predictions)
}

models_result.append(metric_model)

In [20]:
df = pd.DataFrame(models_result)
df.sort_values(by='R2', ascending=False).reset_index(drop=True)
df

,MODEL,MAE,MSE,RMSE,R2
0,RandomForestRegressor+GridSearchCV,0.064109,0.007081,0.084151,0.718451
1,BayesianRidge+GridSearchCV,0.056606,0.005558,0.074549,0.779037
2,GradientBoostingRegressor+GridSearchCV,0.057092,0.005651,0.075174,0.775314
3,MLPRegressor+GridSearchCV,0.056613,0.005560,0.074564,0.778949
